# Librerias usadas

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from surprise import Dataset
from surprise import Reader
from surprise import SVD
from surprise.model_selection import cross_validate
import difflib
import random
import re
from unicodedata import normalize
import unidecode
import nltk

In [157]:
calificaciones.to_csv('../Resultados/calificaciones.csv',encoding="latin-1",sep="|")

# Lectura de datos

In [154]:
mantenimientos = pd.read_excel('../Data/data_20220117.xlsx', sheet_name='PRO-M')
calificaciones = pd.read_excel('../Data/data_20220117.xlsx', sheet_name='Historico calif 2_1(10 años)')

In [155]:
# Solo me quedo con los equipos que tienen mantenimientos y calificaciones
m = calificaciones.Equipo.isin(mantenimientos.Equipo)
calificaciones = calificaciones[m]

# Correccion de la descripcion

In [6]:
mantenimientos['descripcion'] = mantenimientos['Descripción'].str.lower()
mantenimientos['descripcion'] = mantenimientos['descripcion'].apply(unidecode.unidecode)



In [126]:
#mantenimientos['descripcion2'].value_counts().to_csv("../Resultados/mantenimientos_nombres.csv",sep=";")
mantenimientos['descripcion2'].value_counts()

cambiar silica                              98
cambiar rele                                56
sikaflex                                    46
alivio presion                              39
fuga aceite                                 34
                                            ..
revisar contactores-reles-pulsadore oltc     1
revisar alarma buchholz t-tgj01              1
revisar alarma presente del trafo            1
cambiar medidor de temperatura de aceite     1
cambiar gabinete de control reactor rv       1
Name: descripcion2, Length: 164, dtype: int64

In [130]:
palabras_clave = {}
palabras_clave['cambiar silica'] = ['silica revisar','cambiar silica','silica gel','gel cambiar','silica cambiar','silica alivio']
palabras_clave['sikaflex'] = ['sikaflex']
palabras_clave['cambiar rele'] = ['rele alivio','disparo rele','cambiar rele','cambio de rele','rele sello','instalar rele','buchholz cambiar','cambio rele']
palabras_clave['cambiar termometro'] = ['cambiar termometro']
palabras_clave['cambiar valvula'] = ['cambiar valvula']
palabras_clave['cambiar termometro'] = ['cambiar termometro','indicador temperatura','indicador temperatura','indicadores de temperatura']
palabras_clave['fuga aceite'] = ['corregir goteo','mancha aceite','fuga aceite','fuga de aceite','corregir fuga','aceite fuga']
palabras_clave['cambiar indicador nivel de aceite'] = ['cambiar indicador nivel de aceite','indicador nivel de aceite','indicador nivel aceite','indicador nivel aceitefc']
palabras_clave['alivio presion'] = ['alivio presion','presion subita','presion alivio','presion cambiar','valvula alivio']
palabras_clave['cambiar protecciones mecanicas'] = ['mecanicas cambiar','protecciones mecanicas','prot mecanicas','mecanicas medir']
palabras_clave['cambiar aislamient'] = ['aislamient cableado','medir aislamient','bajo aislamiento']

j=0
my_lista=[]
for i in mantenimientos['descripcion']:
    for key,value in palabras_clave.items():
        for word in value:
            if  word in i:
                mantenimientos['descripcion2'][j]=key
                break
            elif mantenimientos['descripcion2'][j]!=mantenimientos['descripcion'][j]:
                mantenimientos['descripcion2'][j]=mantenimientos['descripcion2'][j]
            else:
                mantenimientos['descripcion2'][j]=i
    j=j+1




<ipython-input-130-2e85e9ee7eb9>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mantenimientos['descripcion2'][j]=i
<ipython-input-130-2e85e9ee7eb9>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mantenimientos['descripcion2'][j]=key
<ipython-input-130-2e85e9ee7eb9>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mantenimientos['descripcion2'][j]=mantenimientos['descripcion2'][j]


In [151]:
calificaciones=calificaciones.reset_index()
calificaciones

,level_0,index,Equipo,Denominación,Creado el,Ubicac.técnica,Denominación.1,Posición medida,Valor medido,Unidad caract.,...,Codif.txt.cód.,Fecha,Hora,Hora de,Hora medición,Texto,Texto Ampliado,Creado por,Punto de medida,Documento med.
0,0,2,2011679,Reactor línea monofásico 500 kV-B,2020-12-17,PORC500R23MC-REACT,Calificación del equipo,EQUIPO TOTAL,2,UNIDAD,...,NaN,2020-12-17,21:33:46,00:00:00,21:33:46,/9-OIL.CORR/ /2-INSP=9-ENF/2-PRO-M/ 8-GA,/9-OIL.CORR/ /2-INSP=9-ENF/2-PRO-M/ 8-GAB/8-BUJ/,DDIC,17767894,37658203
1,1,3,2011679,Reactor línea monofásico 500 kV-B,2020-12-11,PORC500R23MC-REACT,Calificación del equipo,EQUIPO TOTAL,2,UNIDAD,...,NaN,2020-12-11,21:41:03,00:00:00,21:41:03,/9-OIL.CORR/ /2-INSP=9-ENF/2-PRO-M/ 8-GA,/9-OIL.CORR/ /2-INSP=9-ENF/2-PRO-M/ 8-GAB/8-BUJ/,DDIC,17767894,37624748
2,2,4,2011679,Reactor línea monofásico 500 kV-B,2020-10-20,PORC500R23MC-REACT,Calificación del equipo,EQUIPO TOTAL,2,UNIDAD,...,NaN,2020-10-20,00:29:20,00:00:00,00:29:20,/9-OIL.CORR/ /2-INSP=9-ENF/2-PRO-M/ 8-GA,/9-OIL.CORR/ /2-INSP=9-ENF/2-PRO-M/ 8-GAB/8-BUJ/,DDIC,17767894,37060963
3,3,5,2011679,Reactor línea monofásico 500 kV-B,2020-10-12,PORC500R23MC-REACT,Calificación del equipo,EQUIPO TOTAL,2,UNIDAD,...,NaN,2020-10-12,21:11:22,00:00:00,21:11:22,/9-OIL.CORR/ /2-INSP=9-ENF/2-PRO-M/ 8-GA,/9-OIL.CORR/ /2-INSP=9-ENF/2-PRO-M/ 8-GAB/8-BUJ/,DDIC,17767894,37034126
4,4,6,2011679,Reactor línea monofásico 500 kV-B,2020-08-02,PORC500R23MC-REACT,Calificación del equipo,EQUIPO TOTAL,2,UNIDAD,...,NaN,2020-08-02,21:11:45,00:00:00,21:11:45,/9-OIL.CORR/ /2-INSP=9-ENF/2-PRO-M/ 8-GA,/9-OIL.CORR/ /2-INSP=9-ENF/2-PRO-M/ 8-GAB/8-BUJ/,DDIC,17767894,36465562
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1066,1066,3594,100008,Reactor línea trifásico 230 kV,2017-01-29,BANA230R12MC-REACT,Calificación del equipo,REACTOR TOTAL,1,UNIDAD,...,NaN,2017-01-29,21:35:25,00:00:00,21:35:25,/1-HUM=DELTA/ /5-INSP=9-ENF/ 8-BUJ/,NaN,DDIC,625629,25904205
1067,1067,3595,100008,Reactor línea trifásico 230 kV,2017-01-13,BANA230R12MC-REACT,Calificación del equipo,REACTOR TOTAL,1,UNIDAD,...,NaN,2017-01-13,21:04:16,00:00:00,21:04:16,/1-HUM=DELTA/ /5-INSP=9-ENF/ 8-BUJ/,NaN,DDIC,625629,25781580
1068,1068,3596,100008,Reactor línea trifásico 230 kV,2016-12-25,BANA230R12MC-REACT,Calificación del equipo,REACTOR TOTAL,1,UNIDAD,...,NaN,2016-12-25,21:26:41,00:00:00,21:26:41,/1-HUM=DELTA/ /2-BUJE//5-INSP=9-ENF/ 8-B,/1-HUM=DELTA/ /2-BUJE//5-INSP=9-ENF/ 8-BUJ/,DDIC,625629,25632380
1069,1069,3597,100008,Reactor línea trifásico 230 kV,2016-12-12,BANA230R12MC-REACT,Calificación del equipo,REACTOR TOTAL,1,UNIDAD,...,NaN,2016-12-12,21:05:49,00:00:00,21:05:49,/1-HUM=DELTA/ /2-BUJE//5-INSP=9-ENF/ 8-B,/1-HUM=DELTA/ /2-BUJE//5-INSP=9-ENF/ 8-BUJ/,DDIC,625629,25553665


In [153]:
rutinas = {} 
rutinas['INSP'] = ['ENF','PRO.M','M.OLTC','CUB','GAB','BUJ.TER']

columnas = ['Equipo','Descripción']
for principal,secundarias in rutinas.items():
    columnas.append(principal)
    for s in secundarias:
        columnas.append(principal + '-' + s)

df_calificaciones = pd.DataFrame(columns = columnas, index = calificaciones.index)

for index in range(len(calificaciones.index)):
    row = calificaciones.loc[index]
    df_calificaciones.loc[index,'Equipo'] = row['Equipo']
    texto = row['Texto']
    texto = texto.replace(' ','')
    texto = texto.replace('_','.')
    texto = texto.replace('-','.')
    if ':' in texto:
        texto = texto[texto.rindex(':') + 1:]
        primer_numerico = re.search("[0-9]",texto)
        texto = texto[primer_numerico.start():]   
    texto_principales = texto.split('/ /')
    for tp in texto_principales:
        #Buscar códigos de las rutinas principales en el texto
        for rutina_principal in rutinas.keys():
            index_inicio_principal = re.search(rutina_principal,tp)
            if not index_inicio_principal == None:
                #Extraer calificación rutina principal
                index_inicio_principal = index_inicio_principal.start()
                index_fin_principal = index_inicio_principal + len(rutina_principal)
                calificacion_principal = tp[index_inicio_principal - 2]
                df_calificaciones.loc[index,rutina_principal] = int(calificacion_principal)
                #Extraer calificaciones secundarias
                if index_fin_principal + 1 < len(tp):
                    texto_secundarias = tp[index_fin_principal + 1:]
                    texto_secundarias = texto_secundarias.split('/')
                    for ts in texto_secundarias:
                        #Buscar códigos de las rutinas secundarias
                        for rutina_secundaria in rutinas[rutina_principal]:
                            if not re.search(rutina_secundaria,ts) == None:
                                calificacion_secundaria = ts[0]
                                if not calificacion_secundaria.isnumeric(): #Si no tiene calificación secundaria, se hereda la calificación principal
                                    calificacion_secundaria = calificacion_principal
                                df_calificaciones.loc[index,rutina_principal + '-' + rutina_secundaria] = int(calificacion_secundaria)
                #continue


AttributeError: 'NoneType' object has no attribute 'start'

In [142]:
range(len(df_diferencias_trazables.index))

TypeError: object of type 'builtin_function_or_method' has no len()

In [134]:


for index in range(len(df_diferencias_trazables.index)):
    row = df_diferencias_trazables.loc[index]
    df_calificaciones.loc[index,'Equipo'] = row['Equipo']
    df_calificaciones.loc[index,'Descripción'] = row['Descripción']
    texto = row['Texto']
    texto = texto.replace(' ','')
    texto = texto.replace('_','.')
    texto = texto.replace('-','.')
    if ':' in texto:
        texto = texto[texto.rindex(':') + 1:]
        primer_numerico = re.search("[0-9]",texto)
        texto = texto[primer_numerico.start():]   
    texto_principales = texto.split('/ /')
    for tp in texto_principales:
        #Buscar códigos de las rutinas principales en el texto
        for rutina_principal in rutinas.keys():
            index_inicio_principal = re.search(rutina_principal,tp)
            if not index_inicio_principal == None:
                #Extraer calificación rutina principal
                index_inicio_principal = index_inicio_principal.start()
                index_fin_principal = index_inicio_principal + len(rutina_principal)
                calificacion_principal = tp[index_inicio_principal - 2]
                df_calificaciones.loc[index,rutina_principal] = int(calificacion_principal)
                #Extraer calificaciones secundarias
                if index_fin_principal + 1 < len(tp):
                    texto_secundarias = tp[index_fin_principal + 1:]
                    texto_secundarias = texto_secundarias.split('/')
                    for ts in texto_secundarias:
                        #Buscar códigos de las rutinas secundarias
                        for rutina_secundaria in rutinas[rutina_principal]:
                            if not re.search(rutina_secundaria,ts) == None:
                                calificacion_secundaria = ts[0]
                                if not calificacion_secundaria.isnumeric(): #Si no tiene calificación secundaria, se hereda la calificación principal
                                    calificacion_secundaria = calificacion_principal
                                df_calificaciones.loc[index,rutina_principal + '-' + rutina_secundaria] = int(calificacion_secundaria)
                #continue


,Equipo,Denominación,Creado el,Ubicac.técnica,Denominación.1,Posición medida,Valor medido,Unidad caract.,Aviso,Codif.txt.cód.,Fecha,Hora,Hora de,Hora medición,Texto,Texto Ampliado,Creado por,Punto de medida,Documento med.
2,2011679,Reactor línea monofásico 500 kV-B,2020-12-17,PORC500R23MC-REACT,Calificación del equipo,EQUIPO TOTAL,2,UNIDAD,NaN,NaN,2020-12-17,21:33:46,00:00:00,21:33:46,/9-OIL.CORR/ /2-INSP=9-ENF/2-PRO-M/ 8-GA,/9-OIL.CORR/ /2-INSP=9-ENF/2-PRO-M/ 8-GAB/8-BUJ/,DDIC,17767894,37658203
3,2011679,Reactor línea monofásico 500 kV-B,2020-12-11,PORC500R23MC-REACT,Calificación del equipo,EQUIPO TOTAL,2,UNIDAD,NaN,NaN,2020-12-11,21:41:03,00:00:00,21:41:03,/9-OIL.CORR/ /2-INSP=9-ENF/2-PRO-M/ 8-GA,/9-OIL.CORR/ /2-INSP=9-ENF/2-PRO-M/ 8-GAB/8-BUJ/,DDIC,17767894,37624748
4,2011679,Reactor línea monofásico 500 kV-B,2020-10-20,PORC500R23MC-REACT,Calificación del equipo,EQUIPO TOTAL,2,UNIDAD,NaN,NaN,2020-10-20,00:29:20,00:00:00,00:29:20,/9-OIL.CORR/ /2-INSP=9-ENF/2-PRO-M/ 8-GA,/9-OIL.CORR/ /2-INSP=9-ENF/2-PRO-M/ 8-GAB/8-BUJ/,DDIC,17767894,37060963
5,2011679,Reactor línea monofásico 500 kV-B,2020-10-12,PORC500R23MC-REACT,Calificación del equipo,EQUIPO TOTAL,2,UNIDAD,NaN,NaN,2020-10-12,21:11:22,00:00:00,21:11:22,/9-OIL.CORR/ /2-INSP=9-ENF/2-PRO-M/ 8-GA,/9-OIL.CORR/ /2-INSP=9-ENF/2-PRO-M/ 8-GAB/8-BUJ/,DDIC,17767894,37034126
6,2011679,Reactor línea monofásico 500 kV-B,2020-08-02,PORC500R23MC-REACT,Calificación del equipo,EQUIPO TOTAL,2,UNIDAD,NaN,NaN,2020-08-02,21:11:45,00:00:00,21:11:45,/9-OIL.CORR/ /2-INSP=9-ENF/2-PRO-M/ 8-GA,/9-OIL.CORR/ /2-INSP=9-ENF/2-PRO-M/ 8-GAB/8-BUJ/,DDIC,17767894,36465562
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3594,100008,Reactor línea trifásico 230 kV,2017-01-29,BANA230R12MC-REACT,Calificación del equipo,REACTOR TOTAL,1,UNIDAD,NaN,NaN,2017-01-29,21:35:25,00:00:00,21:35:25,/1-HUM=DELTA/ /5-INSP=9-ENF/ 8-BUJ/,NaN,DDIC,625629,25904205
3595,100008,Reactor línea trifásico 230 kV,2017-01-13,BANA230R12MC-REACT,Calificación del equipo,REACTOR TOTAL,1,UNIDAD,NaN,NaN,2017-01-13,21:04:16,00:00:00,21:04:16,/1-HUM=DELTA/ /5-INSP=9-ENF/ 8-BUJ/,NaN,DDIC,625629,25781580
3596,100008,Reactor línea trifásico 230 kV,2016-12-25,BANA230R12MC-REACT,Calificación del equipo,REACTOR TOTAL,1,UNIDAD,NaN,NaN,2016-12-25,21:26:41,00:00:00,21:26:41,/1-HUM=DELTA/ /2-BUJE//5-INSP=9-ENF/ 8-B,/1-HUM=DELTA/ /2-BUJE//5-INSP=9-ENF/ 8-BUJ/,DDIC,625629,25632380
3597,100008,Reactor línea trifásico 230 kV,2016-12-12,BANA230R12MC-REACT,Calificación del equipo,REACTOR TOTAL,1,UNIDAD,NaN,NaN,2016-12-12,21:05:49,00:00:00,21:05:49,/1-HUM=DELTA/ /2-BUJE//5-INSP=9-ENF/ 8-B,/1-HUM=DELTA/ /2-BUJE//5-INSP=9-ENF/ 8-BUJ/,DDIC,625629,25553665


In [136]:
df_diferencias_trazables=calificaciones.copy

In [132]:
mantenimientos

,Sociedad CO,Equipo,Denominación,Perfil catálogo,Ubicac.técnica,Indicador ABC,Área de empresa,Pto.tbjo.resp.,Clase de aviso,Aviso,...,Creado el,Inicio deseado,Fin deseado,Prioridad,Fecha de cierre,Status usuario,Status sistema,Autor del aviso,descripcion,descripcion2
0,ISA,202718,Autotrafo Monofásico 230/115 kV-B,N-AUTOTR,LHER230ATR-1-AUTOT,NaN,SUR,38B70001,N4,230230,...,2011-02-16,2011-02-16,2011-06-16,5.0,2011-07-13,CPRO,MECE ORAS,NCALDERON,"fijar tubo valvula aliv.presion, suelto","fijar tubo valvula aliv.presion, suelto"
1,ISA,201232,Autotrafo Monofásico 230/115 kV-A,N-AUTOTR,TORC230ATR-2-AUTOT,NaN,CEN,38B80001,N4,230344,...,2011-02-18,2014-06-01,2014-07-30,2.0,2014-01-31,COPE,MECE ORAS,ISA3970,cambiar silica,cambiar silica
2,ISA,112136,Reactor línea monofásico 500 kV-C,N-REACTO,CERR500R12MC-REACT,NaN,NOR,38B60001,N2,231214,...,2011-03-06,2011-07-01,2011-07-04,5.0,2011-07-19,INIC,MECE ORAS,NBALLESTEROS,corregir fuga aceite purg buchholz rl fc,fuga aceite
3,ISAB,301717,Autotrafo monofasico 230/69/24.9 kV - A,N-AUTOTR,SUCR230ATR-1-AUTOT,NaN,SUC,38F10001,N4,231474,...,2011-03-09,2011-04-06,2011-04-06,2.0,2011-04-29,INIC,MECE ORAS,SVEDIA,cambiar silicagel atr r,cambiar silica
4,ISA,200597,Autotrafo Trifásico 230/115 kV,N-AUTOTR,ESME230ATR-2-AUTOT,NaN,SUR,38B70001,N4,232227,...,2011-03-24,2011-04-20,2011-04-20,2.0,2011-05-06,INIC,MECE ORAS,ERAMOS,cambiar silica gel atr2,cambiar silica
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
538,ISA,120116,Reactor terciario 34.5 kV,1-REAATR,SCAR034R32MC-REACT,C,NOR,38B60001,N2,5548342,...,2021-06-25,2021-07-01,2021-07-01,2.0,2021-07-15,INIC,MECE ORAS,SMARIN,cambiar silica,cambiar silica
539,TRAN,810451,Tebsa 3 60 MVA 220/110/46 kV Fase T,1-TRAAMO,TBS_220ATR03-T,A,TBS,38C10001,N2,5548343,...,2021-06-25,2021-06-30,2021-07-30,2.0,NaT,INIC,METR ORAS,FDELEON,cambiar silica gel,cambiar silica
540,ISA,109355,Reactor línea monofásico 500 kV-A,1-REAAMO,SCAR500R21MC-REACT,C,NOR,38B60001,N2,5548425,...,2021-06-26,2021-06-26,2021-07-25,2.0,NaT,INIC,METR ORAS,SMARIN,cambiar silica,cambiar silica
541,ISA,109356,Reactor línea monofásico 500 kV-B,1-REAAMO,SCAR500R21MC-REACT,C,NOR,38B60001,N2,5548426,...,2021-06-26,2021-06-26,2021-07-25,2.0,NaT,INIC,METR ORAS,SMARIN,cambiar silica,cambiar silica


In [112]:
import re
import unicodedata
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to D:\ambiente\datos de
[nltk_data]     programa\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to D:\ambiente\datos de
[nltk_data]     programa\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


True

In [113]:
ADDITIONAL_STOPWORDS = ['covfefe']

def basic_clean(text):
  """
  A simple function to clean up the data. All the words that
  are not designated as a stop word is then lemmatized after
  encoding and basic regex parsing are performed.
  """
  wnl = nltk.stem.WordNetLemmatizer()
  stopwords = nltk.corpus.stopwords.words('spanish') + ADDITIONAL_STOPWORDS
  text = (unicodedata.normalize('NFKD', text)
    .encode('ascii', 'ignore')
    .decode('utf-8', 'ignore')
    .lower())
  words = re.sub(r'[^\w\s]', '', text).split()
  return [wnl.lemmatize(word) for word in words if word not in stopwords]

In [127]:
words = basic_clean(''.join(str(mantenimientos['descripcion2'].tolist())))

In [129]:
(pd.Series(nltk.ngrams(words, 2)).value_counts())[:60]

(cambiar, silica)              98
(silica, cambiar)              67
(cambiar, rele)                56
(sikaflex, sikaflex)           44
(alivio, presion)              44
(rele, cambiar)                37
(fuga, aceite)                 34
(mecanicas, medir)             29
(medir, aislamient)            28
(aislamient, cableado)         28
(cableado, prot)               28
(prot, mecanicas)              28
(aceite, cambiar)              23
(cambiar, termometro)          22
(termometro, cambiar)          21
(presion, alivio)              18
(cambiar, valvula)             17
(indicador, nivel)             16
(presion, cambiar)             16
(disparo, deshabilitado)       15
(revisar, disparo)             15
(valvula, cambiar)             14
(cambiar, indicador)           13
(aceite, fuga)                 12
(nivel, aceite)                12
(protecciones, mecanicas)       8
(valvula, alivio)               7
(sp, rl)                        7
(deshabilitado, sp)             7
(silica, alivi

In [133]:
calificaciones[]